In [57]:
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import scipy.optimize as spo
import math
from abc import ABC
import natsort

In [152]:
data_path = Path("data_files")
data_folders = [file for file in data_path.iterdir()]
data_folders = natsort.natsorted(data_folders)
test_folder = data_folders[4]
file = test_folder/'seg_1.dat'

In [59]:
class experimental_segment():
    def __init__(self,file) -> None:
        self.parameter_dict = {}
        self.__import_header__(file)

        self.pressure = self.parameter_dict["pressure"]
        self.time_data, self.x_data = np.hsplit(np.genfromtxt(file,delimiter=','),2)
        self.time_data = self.time_data.flatten()
        self.x_data = self.x_data.flatten()
        self.t0 = self.time_data[0]
        self.L0 = self.x_data[0]

        if self.parameter_dict["delta pressure"] > 0:
            self.sign = "pos"
        else: 
            self.sign = "neg"

    def __import_header__(self,file):
        with open(file) as f:
            while True: 
                key = f.readline()
                item = f.readline()

                if key[0] == '#':
                    key = key.strip('#').strip()
                    item = item.strip('#').strip()
                    if 'pressure' in key:
                        item = float(item)
                    self.parameter_dict[key]=item
                else:
                    break

In [60]:
class experiment():
    def __init__(self,dir): 
        seg_files = [file for file in dir.iterdir()]
        seg_files = natsort.natsorted(seg_files)

        self.data_dict = {"pos":[],"neg":[]}

        for file in seg_files:
            segment = experimental_segment(file)
            self.data_dict[segment.sign].append(segment)
        
        if self.data_dict["pos"]:
            self.parameters = self.data_dict["pos"][0].parameter_dict
        else:
             self.parameters = self.data_dict["neg"][0].parameter_dict

        self.parameters.pop("pressure")
        self.parameters.pop("delta pressure")
    
    def 

In [172]:
# Define abstract base class for all models
class Model(ABC):
    # Pass the model class the pressure, initial time, and initial length upon construction
    def __init__(self,seg):
        self.pressure = seg.pressure
        self.t0 = seg.t0
        self.L0 = seg.L0
        self.seg = seg
        self.params = False

    def fit(self):
        (self.params, self.covar) = spo.curve_fit(self.model,self.seg.time_data,self.seg.x_data)
        
    def error(self,k,time,data):
        return np.sum((data-self.model(time,k))**2)
        
    def model(self):
        pass

    def plot(self):
        if not self.params:
            self.fit()
        
        fig,ax = plt.subplots()
        ax.scatter(self.seg.time_data,self.seg.x_data)
        ax.plot(self.seg.time_data,self.model(self.seg.time_data,*self.params),'r')
        plt.show()

class wetting_model(Model):
    def model(self,t,A):
        return np.sqrt(A*(t-self.t0) + np.power(self.L0,2))
        
class wetting_model_modified(Model):
    def model(self,t,A,C):
        return np.sqrt(A*(t-self.t0) + np.power(self.L0,2)) + C


In [166]:
test_seg = experimental_segment(file)
test_model = wetting_model(test_seg)
test_seg2 = experimental_segment(test_folder/'seg_2.dat')
test_model2 = wetting_model(test_seg2)